# Semantiline Kernel

Selles koodinäites kasutate [Semantiline Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI-raamistikku, et luua lihtne agent.

Selle näite eesmärk on näidata samme, mida hiljem kasutame täiendavates koodinäidetes erinevate agentlike mustrite rakendamisel.


## Impordi vajalikud Python'i paketid


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Kliendi loomine

Selles näites kasutame [GitHubi mudeleid](https://aka.ms/ai-agents-beginners/github-models), et pääseda ligi LLM-ile.

`ai_model_id` on määratletud kui `gpt-4o-mini`. Proovi mudelit muuta mõnele teisele GitHubi mudelite turul saadaval olevale mudelile, et näha erinevaid tulemusi.

Selleks, et kasutada `Azure Inference SDK-d`, mida kasutatakse GitHubi mudelite `base_url` jaoks, kasutame Semantic Kernelis `OpenAIChatCompletion` konnektorit. Samuti on olemas teised [saadaval olevad konnektorid](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), et kasutada Semantic Kernelit teiste mudelipakkujate jaoks.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Agendi loomine

Allpool loome agendi nimega `TravelAgent`.

Selles näites kasutame väga lihtsaid juhiseid. Saate neid juhiseid muuta, et näha, kuidas agent erinevalt reageerib.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Agendi käivitamine

Nüüd saame käivitada agendi, määrates lõime tüüpi `ChatHistoryAgentThread`. Kõik vajalikud süsteemisõnumid antakse agendi invoke_stream-i `messages` märksõna argumendina.

Kui need on määratletud, loome `user_inputs`, mis esindab kasutaja poolt agendile saadetavat sõnumit. Antud juhul oleme seadnud selleks sõnumiks `Planeeri mulle päikeseline puhkus`.

Võid vabalt seda sõnumit muuta, et näha, kuidas agent erinevalt reageerib.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
